# 01. TWEETS DATASET

In [128]:
import pandas as pd
from pymongo import MongoClient
import requests
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import json

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/pablofdezc/nltk_data...


1. **We import the dataset from Kaggle, from which we will get the information.**

In [2]:
file = open("../data/realdonaldtrump.csv", encoding="utf8")
data = pd.DataFrame(file)
data.head()

,0
0,"id,link,content,date,retweets,favorites,mentio..."
1,"1698308935,https://twitter.com/realDonaldTrump..."
2,"1701461182,https://twitter.com/realDonaldTrump..."
3,"1737479987,https://twitter.com/realDonaldTrump..."
4,"1741160716,https://twitter.com/realDonaldTrump..."


2. **We clean the dataset keeping the columns we want, arranging columns, dropping null values, removing unuseful columns.**

In [3]:
data[0][0]

'id,link,content,date,retweets,favorites,mentions,hashtags\n'

2.1. **We separate the data from the dataset for every ','.**

In [4]:
data = data[0].str.split(',', expand=True)

In [5]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,id,link,content,date,retweets,favorites,mentions,hashtags\n,None,None,...,None,None,None,None,None,None,None,None,None,None
1,1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,510,917,,\n,None,None,...,None,None,None,None,None,None,None,None,None,None
2,1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,34,267,,\n,None,None,...,None,None,None,None,None,None,None,None,None,None
3,1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,13,19,,\n,None,None,...,None,None,None,None,None,None,None,None,None,None
4,1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,11,26,,\n,None,None,...,None,None,None,None,None,None,None,None,None,None


2.2. **We rename each column with the names we want.**

In [6]:
data.rename(columns={0 : 'id', 1: 'link', 2: 'tweet', 3: 'date', 4: 'rt', 5: 'favourites', 6: 'mentions', 7: 'hashtags'}, inplace=True)

2.3. **We drop the first row which is already setted as column names.**

In [7]:
data = data.drop([0],axis=0)

In [8]:
data

,id,link,tweet,date,rt,favourites,mentions,hashtags,8,9,...,15,16,17,18,19,20,21,22,23,24
1,1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,510,917,,\n,None,None,...,None,None,None,None,None,None,None,None,None,None
2,1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,34,267,,\n,None,None,...,None,None,None,None,None,None,None,None,None,None
3,1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,13,19,,\n,None,None,...,None,None,None,None,None,None,None,None,None,None
4,1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,11,26,,\n,None,None,...,None,None,None,None,None,None,None,None,None,None
5,1773561338,https://twitter.com/realDonaldTrump/status/177...,"""""""My persona will never be that of a wallflow...",2009-05-12 09:07:28,1375,1945,,\n,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43348,1273405198698975232,https://twitter.com/realDonaldTrump/status/127...,Joe Biden was a TOTAL FAILURE in Government. H...,2020-06-17 19:00:32,23402,116377,,\n,None,None,...,None,None,None,None,None,None,None,None,None,None
43349,1273408026968457216,https://twitter.com/realDonaldTrump/status/127...,Will be interviewed on @ seanhannity tonight a...,2020-06-17 19:11:47,11810,56659,@seanhannity,\n,None,None,...,None,None,None,None,None,None,None,None,None,None
43350,1273442195161387008,https://twitter.com/realDonaldTrump/status/127...,pic.twitter.com/3lm1spbU8X,2020-06-17 21:27:33,4959,19344,,\n,None,None,...,None,None,None,None,None,None,None,None,None,None
43351,1273442469066276864,https://twitter.com/realDonaldTrump/status/127...,pic.twitter.com/vpCE5MadUz,2020-06-17 21:28:38,4627,17022,,\n,None,None,...,None,None,None,None,None,None,None,None,None,None


2.4. **We drop the columns we feel are not going to be useful for our analysis.**

In [9]:
data = data.drop(['id', 'link', 'mentions', 'hashtags',8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], axis = 1)
data

,tweet,date,rt,favourites
1,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,510,917
2,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,34,267
3,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,13,19
4,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,11,26
5,"""""""My persona will never be that of a wallflow...",2009-05-12 09:07:28,1375,1945
...,...,...,...,...
43348,Joe Biden was a TOTAL FAILURE in Government. H...,2020-06-17 19:00:32,23402,116377
43349,Will be interviewed on @ seanhannity tonight a...,2020-06-17 19:11:47,11810,56659
43350,pic.twitter.com/3lm1spbU8X,2020-06-17 21:27:33,4959,19344
43351,pic.twitter.com/vpCE5MadUz,2020-06-17 21:28:38,4627,17022


2.5. **We now set the date column to a proper format. We want only the year of the tweet.**

In [14]:
date_year = data.date.apply(lambda x: str(x) [0:4])

In [17]:
df = pd.DataFrame(date_year)
df = df.rename(columns={'date': 'year'})
df

,year
1,2009
2,2009
3,2009
4,2009
5,2009
...,...
43348,2020
43349,2020
43350,2020
43351,2020


In [18]:
data = pd.concat([data, df], axis=1).copy()
data.head()

,tweet,date,rt,favourites,year
1,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,510,917,2009
2,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,34,267,2009
3,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,13,19,2009
4,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,11,26,2009
5,"""""""My persona will never be that of a wallflow...",2009-05-12 09:07:28,1375,1945,2009


2.6. **We now check that there aren't any null values. If there are any, we will remove them.**

In [132]:
data.isnull().sum()

tweet         0
date          0
rt            0
favourites    0
year          0
dtype: int64

In [105]:
data.columns


Index(['tweet', 'date', 'rt', 'favourites', 'year'], dtype='object')

3. **We now save our new and clean dataframe into a csv file, convert it to a json and then, export it to MongoDB.**

In [21]:
data.to_csv('../data/dt_tweets.csv')

In [41]:
data.to_json("../data/dtt_script_json", orient="records")

4. **Connect to MongoDB**

In [45]:
#mongoimport --db DT --collection tweets --jsonArray dtt_script_json

In [133]:
client = MongoClient("localhost:27017")

In [134]:
client.list_database_names()

['DT', 'HP', 'Ironhack', 'admin', 'config', 'local']

In [135]:
db = client.get_database("DT")

In [137]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'DT')

In [138]:
db.list_collection_names()

['tweets']

In [139]:
collection = db.get_collection("tweets")

In [140]:
collection.find_one()

{'_id': ObjectId('608c4e00673c051e14d20d78'),
 '': '1',
 'tweet': 'Be sure to tune in and watch Donald Trump on Late Night with David Letterman as he presents the Top Ten List tonight!',
 'date': datetime.datetime(2009, 5, 4, 11, 54, 25),
 'rt': 510,
 'favourites': 917,
 'year': 2009}

## QUERYS

#### Query one - number of retweets of each tweet to see the popularity of them

In [141]:
collection.find_one({"rt": {"$lt": 920}})

{'_id': ObjectId('608c4e00673c051e14d20d78'),
 '': '1',
 'tweet': 'Be sure to tune in and watch Donald Trump on Late Night with David Letterman as he presents the Top Ten List tonight!',
 'date': datetime.datetime(2009, 5, 4, 11, 54, 25),
 'rt': 510,
 'favourites': 917,
 'year': 2009}

In [142]:
rt = collection.find({'rt':{"$gt": 200000}}, {"tweet": 1, "rt": 1, "date":1, "_id" : 0}).sort("date", -1)
list(rt)

[{'tweet': 'The United States of America will be designating ANTIFA as a Terrorist Organization.',
  'date': datetime.datetime(2020, 5, 31, 9, 23, 43),
  'rt': 224890},
 {'tweet': '# FraudNewsCNN # FNNpic.twitter.com/WYUnHjjUjg',
  'date': datetime.datetime(2017, 7, 2, 6, 21, 42),
  'rt': 302269},
 {'tweet': 'TODAY WE MAKE AMERICA GREAT AGAIN!',
  'date': datetime.datetime(2016, 11, 8, 4, 43, 14),
  'rt': 289872},
 {'tweet': 'Are you allowed to impeach a president for gross incompetence?',
  'date': datetime.datetime(2014, 6, 4, 3, 23, 11),
  'rt': 242667}]

#### Query two - number of favourites of each tweet to see the importance of it to the followers

In [143]:
rt = collection.find({'favourites':{"$gt": 800000}}, {"tweet": 1, "favourites": 1, "_id" : 0})
list(rt)

[{'tweet': 'The United States of America will be designating ANTIFA as a Terrorist Organization.',
  'favourites': 828937}]

#### Query three - year of the tweet

In [150]:
year = collection.find({'year':{"$gt": 2019}}, {"tweet": 1, "year": 1, "_id" : 0}).limit(20)
list(year)

[{'tweet': 'Best equipment & finest military in the World. On site quickly!https://twitter.com/heatherjones333/status/1212475089133944832 …',
  'year': 2020},
 {'tweet': 'Great job!https://twitter.com/TrumpWarRoom/status/1212391791774838784 …',
  'year': 2020},
 {'tweet': 'They don’t know how to do that!https://twitter.com/GeraldoRivera/status/1212402251806990336 …',
  'year': 2020},
 {'tweet': 'pic.twitter.com/VXeKiVzpTf', 'year': 2020},
 {'tweet': 'pic.twitter.com/qOi7mpKcHY', 'year': 2020},
 {'tweet': '95% Approval Rating in the Republican Party. Thank you!',
  'year': 2020},
 {'tweet': 'IRAN WILL NEVER HAVE A NUCLEAR WEAPON!', 'year': 2020},
 {'tweet': 'pic.twitter.com/XpVY2bnR5a', 'year': 2020},
 {'tweet': 'pic.twitter.com/dXBP7Srsqy', 'year': 2020},
 {'tweet': 'Amazing analysis & reporting!https://twitter.com/realDonaldTrump/status/1214230745176322048 …',
  'year': 2020},
 {'tweet': 'We love Australia!https://twitter.com/ScottMorrisonMP/status/1214373985988698112 …',
  'year': 20

#### Query four - polarity of the tweet to see how positive or negative they are

In [160]:
def avg_polarity2(year):
    query_fourth = {'year': year}
    year = list(collection.find(query_fourth, {"tweet": 1, "_id" : 0}))

    sia = SentimentIntensityAnalyzer()

    pos = []
    for i in year:
        polarity = sia.polarity_scores(i['tweet'])
        pos.append(polarity['pos'])

    mean_anual_polarity = sum(pos)/len(pos)    
    print(mean_anual_polarity)
    return mean_anual_polarity

In [164]:
def avg_polarity(year):
    mean_anual_polarity = avg_polarity2(year)
    return json.dumps(mean_anual_polarity)
avg_polarity("2019")

ZeroDivisionError: division by zero

### No he podido sacar la formula bien para que me haga la polaridad de los tweets, no se que falla

<!-- ## Query -->